In [ ]:
#!pip install pandas keras numpy xgboost sklearn nltk textblob

In [ ]:
#!conda install pytorch torchvision -c pytorch

## Import Data and set Test Train Splits

In [1]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas as pd, xgboost, numpy as np, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
import os

real_users = pd.read_csv('/home/cheriexu/genuine_accounts/users.csv',low_memory = False)
real_tweets = pd.read_csv('/home/cheriexu/genuine_accounts/tweets.csv',low_memory = False)
real_df = real_tweets.merge(real_users, on='id',how = 'outer', suffixes = ('_tweets','_users')) #Merging the user and tweets
real_df['labels']=pd.Series(np.ones(len(real_df['id']))) #Creating a new column to indicate that this is human data
#Get Bot data 
directory = os.fsencode('/home/cheriexu/russian-troll')
troll_data = pd.DataFrame()
for file in os.listdir(directory):
  print(file)
  filename = os.fsdecode(file)
  if filename.endswith(".csv"):
    botdata = pd.read_csv('/home/cheriexu/russian-troll/'+filename,low_memory = False)
    troll_data = troll_data.append(botdata) #append together
troll_data = troll_data.rename(index = str, columns = {"publish_date":"date"})


/home/shared/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


b'IRAhandle_tweets_9.csv'
b'IRAhandle_tweets_4.csv'
b'IRAhandle_tweets_8.csv'
b'IRAhandle_tweets_7.csv'
b'IRAhandle_tweets_11.csv'
b'IRAhandle_tweets_10.csv'
b'IRAhandle_tweets_3.csv'
b'IRAhandle_tweets_6.csv'
b'IRAhandle_tweets_12.csv'
b'IRAhandle_tweets_2.csv'
b'IRAhandle_tweets_1.csv'
b'README.md'
b'IRAhandle_tweets_13.csv'
b'IRAhandle_tweets_5.csv'


In [2]:
frac = 50       #Factor to Downsample due to runtime constraints 
troll_data['labels']=0
#Limiting too only English language tweets
english_bot = troll_data[troll_data['language']  == 'English']
troll_data['language'].unique()
print(len(english_bot),len(troll_data)) #Class imbalance though, we need more spam bot results
train_df = pd.DataFrame() #Create train data frame for splitting later
real_df = real_df.rename(index = str, columns = {"timestamp_tweets": "date"})
real= real_df[['text','labels','date']].dropna()
train_df = train_df.append(real[['text','labels','date']].sample(int(len(real_df)/frac)))
english_bot = english_bot.rename(index=str, columns={"content": "text"})
e_b = english_bot[['text','labels','date']].dropna()
train_df = train_df.append(e_b[['text','labels','date']].sample(int(len(english_bot)/frac)))
len(train_df)

2116867 2946207


99193

## Preprocess Cells and Clean Text


In [3]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from nltk.stem.snowball import SnowballStemmer
# Others
import nltk
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
import re
from sklearn.manifold import TSNE

In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/cheriexu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
def clean_text(text):
    ## Remove puncuation
    text = str(text)
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    ## Stemming
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)
    return text

train_df['text'] = train_df['text'].map(lambda x: clean_text(x))

In [6]:
train_df['labels'].unique()

array([1., 0.])

In [7]:
#Create Vocab and Create Tokenized array
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(train_df['text'])
sequences = tokenizer.texts_to_sequences(train_df['text'])
data = pad_sequences(sequences, maxlen=50)

In [8]:
encoder = preprocessing.LabelEncoder()
y = encoder.fit_transform(train_df['labels'])

#valid_y = encoder.fit_transform(valid_y)
#np.array(train_df['labels'])

In [9]:
model = Sequential()
model.add(Embedding(20000, 100, input_length=50))
model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
## Fit the model
model.fit(data, y, validation_split=0.4, epochs=3)

Train on 59515 samples, validate on 39678 samples
Epoch 1/3
29856/59515 [==============>...............] - ETA: 1:17 - loss: 0.1078 - acc: 0.9698

KeyboardInterrupt: 

In [15]:
#with Conv 
def create_conv_model():
    model_conv = Sequential()
    model_conv.add(Embedding(20000, 100, input_length=50))
    model_conv.add(Dropout(0.2))
    model_conv.add(Conv1D(64, 5, activation='relu'))
    model_conv.add(MaxPooling1D(pool_size=4))
    model_conv.add(LSTM(100,dropout=0.3, recurrent_dropout=0.2))
    model_conv.add(Dense(1, activation='sigmoid'))
    model_conv.compile(loss='binary_crossentropy', optimizer='adam',    metrics=['accuracy'])
    return model_conv
model_conv = create_conv_model()
model_conv.fit(data, y, validation_split=0.2, epochs = 2)

Train on 79354 samples, validate on 19839 samples
Epoch 1/2
79354/79354 [==============================] - 100s 1ms/step - loss: 0.2812 - acc: 0.8944 - val_loss: 0.5275 - val_acc: 0.7836
Epoch 2/2
79354/79354 [==============================] - 101s 1ms/step - loss: 0.1938 - acc: 0.9311 - val_loss: 0.5433 - val_acc: 0.7919


In [19]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(train_df['text'], y, test_size = .2)
# Create CountVectorizer
count_vect = CountVectorizer(stop_words = 'english',analyzer='word', token_pattern=r'\w{1,}')
#fill in nan to make sklearn happy


count_vect.fit(train_df['text'])
# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [20]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    classifier.fit(feature_vector_train, label)
    pred_train = classifier.predict(feature_vector_train)
    predictions = classifier.predict(feature_vector_valid)
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)

    return metrics.classification_report(label, pred_train), metrics.classification_report(valid_y, predictions)#confusion matrix on validation matrix
  #metrics.confusion_matrix(label, pred_test), metrics.confusion_matrix(valid_y, predictions)

In [21]:
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)
valid_y

array([0, 1, 1, ..., 1, 1, 1])

In [22]:
# Naive Bayes on Count Vectors
np.nan_to_num(xtrain_count)
np.nan_to_num(xvalid_count)
conf_test, conf_valid = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
print (conf_test[0:2])
print(conf_valid[0:2])

'''

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print "NB, N-Gram Vectors: ", accuracy

# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
'''

'\n\n# Naive Bayes on Ngram Level TF IDF Vectors\naccuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)\nprint "NB, N-Gram Vectors: ", accuracy\n\n# Naive Bayes on Character Level TF IDF Vectors\naccuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)\n'

In [23]:
conf_test

'             precision    recall  f1-score   support\n\n          0       0.93      0.93      0.93     33859\n          1       0.95      0.95      0.95     45495\n\navg / total       0.94      0.94      0.94     79354\n'